<a href="https://colab.research.google.com/github/halen48/VisualG2C/blob/main/VisualG2C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [321]:
#@title Dependências
import re
import inspect
from google.colab import files
import os
import shutil

In [322]:
modo = 'Upload' #@param ['Upload', 'Exemplo']
sufixo_arquivo_c = ".c" #@param
sufixo_arquivo_visgualg = ".alg" #@param
pasta_dos_exercicios = "exercicios" #@param


#Programa

In [323]:
programas_exemplos = []

In [324]:
programas_exemplos.append("""
algoritmo "Calculos de numeros"
// Função : Exibe informações como o maior, menor número, a média e a sua soma
// Autor : Guilherme Novaes
// Data : 11/03/2022
// Seção de Declarações
var
numeros: vetor [1..5] de inteiro
teste1,teste2,teste3, teste4 : vetor [1..3,1..4] de real
nome : caractere
max, min,i,soma:inteiro
media : real
inicio
// Seção de Comandos

escreva("Digite seu nome: ")
leia(nome)

max<-numeros[1];
min<-numeros[1];
para i de 1 ate 5 faca
  escreva (nome, ", digite o ", i,"o valor: ")
  leia(numeros[i])

  // soma dos valores digitados
  soma <- soma + numeros[i];
  //media entre os valores digitados
  media<- soma / 5;

  se (numeros[i] > max) entao
    max<- numeros[i];
  fimse

  se (numeros[i] < min) entao
    min <- numeros[i];
  fimse
fimpara

escreval ("maior numero: ", max)
escreval ("menor numero: ",min)
escreval("Soma dos numeros digitados:",soma)
escreva ("Média dos numero digitados:", media)

fimalgoritmo
""")

In [325]:
programas_exemplos.append("""
algoritmo "Times"
var time: caractere
inicio
escreva ("Entre com o nome de um time de futebol: ")
leia (time)
escolha time
caso "Flamengo", "Fluminense", "Vasco", "Botafogo"
escreval ("É um time carioca.")
caso "São Paulo", "Palmeiras", "Santos", "Corínthians"
escreval ("É um time paulista.")
outrocaso
escreval ("É de outro estado.")
fimescolha
fimalgoritmo
""")

In [326]:
programas_exemplos.append("""
algoritmo "Números de 1 a 10 (com interrompa)"
var x: inteiro
inicio
x <- 0
repita
x <- x + 1
escreva (x:3)
se x = 10 entao
interrompa
fimse
ate falso
fimalgoritmo
""")

#Parse

##Funções de Parse e Constantes

In [327]:
palavras_chave = {
    "inicio" : "fimalgoritmo",
    "se" : 'fimse',
    "para" : 'fimpara',
    "enquanto" : 'fimenquanto',
    "escolha" : "fimescolha",
    'repita' : 'ate',
}

In [328]:
palavras_chave_fechamento = {}
for palavra in palavras_chave:
  palavras_chave_fechamento[palavras_chave[palavra]] = palavra

In [329]:
tipos_variaveis = {
    'inteiro' : 'int',
    'real' : 'float',
    'caractere' : 'char',
    'logico' : 'bool',
}

In [330]:
mascaras_tipos_c = {
    'int' : '%d',
    'float' : '%f',
    'char' : '%s',
    'bool' : '%d',
}

```
Tamanho máximo da string que o visualG suporta
```

In [331]:
max_tam_string = 500 #@param {type:"integer"}

###Sub-parces

###Apelidos: Utilizados em um simples replace

In [332]:
apelidos = {
    '=' : '==',
    '<>' : '!=',
    '\\' : '/',
    ' mod ' : ' % ',
    ' div ' : ' / ',
    '<-' : '=',
    'limpatela' : '',
    'falso': 'false',
    'verdadeiro' : 'true',
    'arcsen' : 'asen',
    'arccos' : 'acos',
    'arctan' : 'atan',
    'cotan' : 'ctan',
    'logn' : 'log',
    'raizq' : 'sqrt',
    'exp' : 'pow',
    'caracpnum' : 'atoi',
    'numpcarac' : 'itoa',
    'compr'  : 'strlen',
    'copia'  : 'strcpy',
    'maiusc' : 'upper',
    'minusc' : 'lower',
    'abs' : 'abs',
}

###Regex

In [333]:
vetor_mask = re.compile(r'(.*): *vetor *\[(.*)\] * de * (\w+)')
var_mask = re.compile(r'(.*) *: *(\w+)')
potencia_mask = re.compile(r'([^\^]+)\^([^\-\+\*\\]+)') #pow([0],[1])
parametros_mask = re.compile(r'\((.*)\)')
para_mask = re.compile(r"para (.*) de (.*) ate (.*) faca")
passo_mask = re.compile(r'(.*) passo (.*)')
se_mask = re.compile(r"se *(.*) *entao")
enquanto_mask = re.compile(r"enquanto *(.*) *faca")
string_mask = re.compile(r'\"([^\"]+)\"')

###Utilidades

In [334]:
def eh_funcao(linha, parenteses = True):
  func_name = inspect.stack()[1].function.replace('parse_','')
  if(parenteses):
    try:
      params = parametros_mask.findall(linha)[0]
    except:
      raise Exception("[%s]: Erro de sintaxe em <%s>"%(func_name,linha))
  else:
    params = linha[len(func_name):]
  return (func_name == linha[:len(func_name)].lower()),func_name, params

In [335]:
def nome_variavel(nome):
  idx = nome.find('[')
  if(idx > 0):
    return nome[:idx]
  return nome

###Casos especiais

####Base

In [336]:
def parse_inicio(**kwargs):
  t_vars, n_vars, linha = kwargs['t_vars'], kwargs['n_vars'], kwargs['linha']
  eh,_,_ = eh_funcao(linha, False)
  if(not eh):
    return None
  
  ret = """
#include <stdio.h>
#include <stdlib.h>
#include <stdbool.h>
#include <string.h>
#include <math.h>
#include <time.h>
#include <ctype.h>

#define PI 3.141592
\n
\n
void upper(char * v){ int i; for(i = 0; i < strlen(v); i++) v[i] = toupper(v[i]); }
void lower(char * v){ int i; for(i = 0; i < strlen(v); i++) v[i] = tolower(v[i]); }
\n
int main() {
\tint loop__var___;
\tsrand(time(NULL));
\n
\n
"""

  for tipo in t_vars:
    ret += '\t'+tipo + ' '
    for i in range(0,len(t_vars[tipo]), 5):
       ret += ', '.join(t_vars[tipo][i:i+5])
    ret += ';\n'
  
  ret += '\n'.join(['\tmemset(&%s, 0, sizeof(%s));'%(nome,nome) for nome in n_vars])

  return [ret+'\n']

In [337]:
def parse_fimalgoritmo(**kwargs):
  linha = kwargs['linha']
  eh,_,_ = eh_funcao(linha, False)
  if(not eh):
    return None
  return ["""
\treturn 0;
}
"""]

####Entrada & Saída

In [338]:
def parse_escreva(**kwargs):
  linha, n_vars, ignorar_string = kwargs['linha'], kwargs['n_vars'], kwargs['ignorar_string']
  eh,nome,params = eh_funcao(linha)
  if(not eh):
    return None
  pula_linha = ('escreval' == linha[:len('escreval')])
  printf_params = ""
  vars = []

  #primeiro isola as strings
  strs = string_mask.findall(params)
  for i,s in enumerate(strs):
    params = params.replace(s,'{%d}'%i)
  
  mask = None
  #agora realiza as outras operacoes
  for param in params.split(','):
    param = param.strip()
    if(not len(param)):
      param = ' '
    else:
      nome = nome_variavel(param)
      if(':' in nome):
        nome = nome.split(':')
        nome,mask = nome[0],'.'.join(nome[1:])
        idx = param.find(':')
        param = param[:idx]
    if(nome in n_vars):
      if(mask):
        printf_params += '%'+mask+mascaras_tipos_c[n_vars[nome]].replace('%','')
      else:
        printf_params += mascaras_tipos_c[n_vars[nome]]
      vars.append(param)
    elif(param[0] == '"'):
      printf_params += (' ' if(ignorar_string) else strs[int(param[2:-2])])
  
  if(pula_linha):
    printf_params += '\\n'

  printf_params += '"'
  if(vars):
    printf_params += ', '+', '.join(vars)
  
  return [('printf("%s);'%printf_params)]

In [339]:
parse_escreval = parse_escreva

In [340]:
def parse_leia(**kwargs):
  linha, n_vars = kwargs['linha'], kwargs['n_vars']
  eh,_,params = eh_funcao(linha)
  if(not eh):
    return None
  
  scanf_params = ""
  vars = []

  for param in params.split(','):
    param = param.strip()
    nome = nome_variavel(param)
    if(nome in n_vars):
      scanf_params += mascaras_tipos_c[n_vars[nome]]
      
      if(n_vars[nome] == 'char'):
        vars.append(param)
      else:
        vars.append("&"+param)
    else:
      print("Variáveis do programa:")
      print(n_vars)
      raise Exception("[Leia]: Variavel %s não existe!!"%param)
  

  scanf_params += '"'
  if(vars):
    scanf_params += ', '+', '.join(vars)
  
  return [('scanf("%s);'%scanf_params)]

####Condicional

In [341]:
switches = []

In [342]:
def parse_escolha(**kwargs):
  global switches
  linha, n_vars = kwargs['linha'], kwargs['n_vars']
  eh,_,_ = eh_funcao(linha, False)
  if(not eh):
    return None
  switches.append({"Var" : None, "Tipo" : None, "LastOpen" : False})
  switch = switches[-1]
  switch["Var"] = linha.split(' ')[-1].strip()
  if(not switch["Var"] in n_vars):
    raise Exception("[Escolha]: Variável <%s> não encontrada!"%switch['Var'])
  switch["Tipo"] = n_vars[switch["Var"]]
  if(switch["Tipo"] == 'int'):
    return ["switch(%s)"%switch["Var"]]
  elif(switch["Tipo"] == 'float'):
    raise Exception("[Escolha]: Variável <%s> não pode ser do tipo real!!"%switch['Var'])
  else:
    return ["\n"]

In [343]:
def parse_caso(**kwargs):
  global switches
  linha = kwargs['linha']
  eh,nome,params = eh_funcao(linha, False)
  if(not eh):
    return None
  switch = switches[-1]
  cases = params.split(',')
  ret = ""
  # ====================================
  if(switch["LastOpen"]):
    if(switch["Tipo"] == 'int'):
      ret = 'break;'
    else:
      ret = '} else '
  else:
    switch["LastOpen"] = True
  # ====================================
  if(switch["Tipo"] == 'int'):
    return [ret+"\n",'\n'.join(["case %s:"%case for case in cases])]
  else:
    return [ret+'if(%s) {'%(' || '.join(['(memcmp(%s,%s,sizeof(%s)) == 0)'%(switch['Var'], case, case) for case in cases]))]

In [344]:
def parse_outrocaso(**kwargs):
  global switches
  linha = kwargs['linha']
  eh,_,_ = eh_funcao(linha, False)
  if(not eh):
    return None
  switch = switches[-1]
  if(switch['Tipo'] == 'int'):
    return ['default:']
  else:
    return ['} else {']

In [345]:
def parse_se(**kwargs):
  linha = kwargs['linha']
  eh,_,_ = eh_funcao(linha, False)
  if(not eh):
    return None
  try:
    exp = se_mask.findall(linha)[0]
  except:
    raise Exception("[Se]: Erro de sintaxe em <%s>"%linha)

  return [('if (%s) {'%(exp))]

####Laço

In [346]:
def parse_para(**kwargs):
  linha, n_vars = kwargs['linha'], kwargs['n_vars']
  eh,_,_ = eh_funcao(linha, False)
  if(not eh):
    return None
  
  try:
    var, inicio, fim = para_mask.findall(linha)[0]
  except:
    raise Exception("[Para]: Erro de sintaxe em <%s>"%linha)
  
  if('passo' in fim):
    try:
      fim, passo = passo_mask.findall(fim)[0]
    except:
      raise Exception("[Para]: Erro de sintaxe em <%s>"%linha)
  else:
    passo = 1

  if(not nome_variavel(var) in n_vars):
    raise Exception ("[Para]: <%s> não declarada!!"%var)

  #Aparentemente no visual G é algo assim. Se passo for 0, o visual G emite um erro...
  return [('for(%s = %s, loop__var___ = 0; loop__var___ <= (%s-%s/%s); %s+=%s, loop__var___+=%s) {'%(var, inicio, fim, inicio, passo, var, passo, passo))]

In [347]:
def parse_ate(**kwargs):
  linha, n_vars = kwargs['linha'], kwargs['n_vars']
  eh,nome,params = eh_funcao(linha, False)
  if(not eh):
    return None
  
  vars = {}
  for name in n_vars:
    vars[name] = 0
  try:
    eval(params.replace('false','False').replace('true','True'),{'__builtins__': None},vars)
  except:
    pass
    #raise Exception("[Ate]: Erro de sintaxe em <%s> (Variável não existe?)"%linha)

  return [('} while(!(%s));'%(params))]

In [348]:
def parse_interrompa(**kwargs):
  linha = kwargs['linha']
  eh,_,_ = eh_funcao(linha, False)
  if(not eh):
    return None
  return ['break;']

In [349]:
def parse_repita(**kwargs):
  linha = kwargs['linha']
  eh,_,_ = eh_funcao(linha, False)
  if(not eh):
    return None
  return ['do {\n']

In [350]:
def parse_enquanto(**kwargs):
  linha = kwargs['linha']
  eh,_,_ = eh_funcao(linha, False)
  if(not eh):
    return None

  try:
    exp = enquanto_mask.findall(linha)[0]
  except:
    raise Exception("[Enquanto]: Erro de sintaxe em <%s>"%linha)

  return [('while (%s) {'%(exp))]

####Matemática

In [351]:
def parse_randi(**kwargs):
  eh,_,params = eh_funcao(kwargs['linha'])
  if(not eh):
    return None
  return ['(rand()%%%s)'%params]

In [352]:
def parse_rand(**kwargs):
  eh,_,_ = eh_funcao(kwargs['linha'], False)
  if(not eh):
    return None
  return ['((rand()%32767)/32767.)']

In [353]:
def parse_int(**kwargs):
  eh,_,params = eh_funcao(kwargs['linha'])
  if(not eh):
    return None
  return ['((int)(%s))'%params]

In [354]:
parse_asc = parse_int

In [355]:
def parse_grauprad(**kwargs):
  eh,_,params = eh_funcao(kwargs['linha'])
  if(not eh):
    return None
  return ['(%s * (PI/180.))'%params]

In [356]:
def parse_asc(**kwargs):
  eh,_,params = eh_funcao(kwargs['linha'])
  if(not eh):
    return None
  return ['(%s * (PI/180.))'%params]

###Outras funções

In [357]:
def limpa_comentario(codigo):
  codigo_final = ""

  ignora = False
  for idx in range(len(codigo)):
    if(ignora):
      if(codigo[idx] == '\n'):
        ignora = False
    else:
      if(codigo[idx:idx+2] == '//'):
          ignora = True
      else:
        codigo_final += codigo[idx]

  return codigo_final.replace('\t',' ')

In [358]:
def variaveis_globais(codigo):
  #=====================================================================
  def var_dict(v):
    if(not v in t_vars):
        t_vars[v] = []
  #=====================================================================
  t_vars = {} #tipo : nome
  n_vars = {} #nome : tipo
  escopo = codigo[:codigo.lower().find('inicio')]
  
  for nomes_var, tipo in var_mask.findall(escopo):
  # ============ Variaveis simples =====================================
    nomes_var, tipo = nomes_var.lower().replace('var ',''), tipo.lower()
    if(tipo == 'vetor'):
      continue #Tradados depois...
    var_dict(tipos_variaveis[tipo])
    #Tipo caractere no visual G é uma string
    if(tipo == 'caractere'):
      t_vars[tipos_variaveis[tipo]] += [var.strip()+'[%d]'%max_tam_string for var in nomes_var.split(',')]
    else:
      t_vars[tipos_variaveis[tipo]] += [var.strip() for var in nomes_var.split(',')]
  # ============= Vetores ==============================================
  for nomes_var, dimensoes, tipo in vetor_mask.findall(escopo):
    tipo = tipo.lower()
    #No visualG, vetores começam na posição 1, e não na 0, logo, sempre será alocado uma posição adicional...
    dimensoes = ''.join(['[%d]'%(int(v.split('..')[-1]) + 1) for v in dimensoes.split(',')])
    var_dict(tipos_variaveis[tipo])
    t_vars[tipos_variaveis[tipo]] += ["%s%s"%(v.strip().lower(),dimensoes) for v in nomes_var.split(',')]
    
  
  for tipo in t_vars:
    for nome in t_vars[tipo]:
      idx = nome.find('[')
      if(idx > 0):
        n_vars[nome[:idx]] = tipo
      else:
        n_vars[nome] = tipo
  
  return t_vars,n_vars

###Classe principal

In [359]:
class VisualG2C():
  def __init__(self, programa):
    self.__stack = []
    self.__programa = limpa_comentario(programa)
    self.t_vars,self.n_vars = variaveis_globais(self.__programa)

  def __parse_bloco(self, lin, ignorar_string):
    lin = lin.strip()
    ret = None
    tokens = (
        lin
        .lower()
        .replace('(', ' ( ')
        .replace(')', ' ) ')
        .split(' ')
    )
    
    try:
      if('[' in tokens[0]):
        raise NameError #Array
      ret = eval("parse_%s(linha = lin.strip(), n_vars = self.n_vars, t_vars = self.t_vars, ignorar_string = ignorar_string)"%tokens[0].lower())
    except NameError:
      ret = None
    nv = len(self.__stack)
    for token in tokens:
      if(token == 'ate' and 'para' in tokens):
        continue #Ignora esse...
      if(token in palavras_chave_fechamento):
        if(palavras_chave[self.__stack[-1]] == token):
          self.__stack.pop()
          nv -= 1
        else:
          print(self.__stack)
          print(tokens)
          raise Exception("[Bloco] Erro no delimitador <%s>"%token)
      if(token in palavras_chave):
        self.__stack.append(token)
    
    
    if(not ret and lin in palavras_chave_fechamento):
      ret = '}'
    return nv,ret
      
  '''
  eval(self, ignorar_string = False)
  @param ignorar_string: Caso seja True, é construido um programa em C removendo
                         todas as strings das funções escreva e escreval, deixando
                         apenas print de variáveis separadas por ' '
  @param mostra_program: Mostra o programa convertido em C na tela
  '''

  def eval(self, ignorar_string = False, mostra_programa = False):
    
    if(self.__programa.lower().find('funcao') >= 0 or self.__programa.lower().find('procedimento') >= 0):
      print("Funcao e Procedimento ainda não possuem parse!")
      return ""

    
    idx = self.__programa.lower().find('inicio')
    
    if(idx < 0):
      raise Exception("[Eval]: Programa não possui <inicio>!!!")

    for valor in apelidos:
      if(apelidos[valor].isalpha()):
        self.__programa =  re.sub(valor, " %s "%apelidos[valor], self.__programa, flags=re.IGNORECASE)
      else:
        self.__programa = self.__programa.replace(valor, " %s "%apelidos[valor])

    programa_C = [""]
    
    for lin in self.__programa[idx:].split('\n'):
      nv, n_lins = self.__parse_bloco(lin, ignorar_string)
      
      if(n_lins is None):
        programa_C.append('\t'*(nv))
        programa_C[-1] += (lin)
        if(' = ' in lin and programa_C[-1][-1] != ';'):
          programa_C[-1] += ';'
      else:
        for p in n_lins:
          programa_C.append('\t'*(nv))
          
          if(('caso' in lin.lower())):
            programa_C[-1] += len(switches)*'\b'
          programa_C[-1] += (p)
          
      
    if(len(self.__stack)):
      print(self.__stack)
      raise Exception("[Eval]: Erro de sintaxe em <%s>!!!"%self.__stack[-1])

    if(mostra_programa):
      for linha in programa_C:
        print(linha) 

    programa_C = '\n'.join(programa_C).replace('\b','')

    return programa_C



#Exemplo

In [360]:
if(modo == 'Demonstração'):
  for i,programa in enumerate(programas_exemplos):
    p = VisualG2C(programa)
    with open(("%03d"%i)+sufixo_arquivo_c, 'w') as f:
      f.write(p.eval(mostra_programa = True))
    print("Programa %03d salvo!"%i)


In [361]:
if(modo == 'Demonstração'):
  for i in range(len(programas_exemplos)):
    f_name = ("%03d"%i)+sufixo_arquivo_c
    !gcc $f_name -o teste
    print('\n')
    print("="*40)
    !./teste

#Execuções

In [362]:
try:
  shutil.rmtree(pasta_dos_exercicios)
except OSError:
  pass
try:
  os.mkdir(pasta_dos_exercicios)
except FileExistsError:
  pass

In [363]:
mask_aluno = re.compile(r'([^_]+)_\d+__assignsubmission_file_')

In [364]:
#@title Submeta um zip com seus arquivos
if(modo == 'Upload'):
  upload = files.upload() 
  for arq in upload:
    arq = '"%s"'%arq
    !unzip $arq -d $pasta_dos_exercicios

Saving algoritmos.zip to algoritmos (5).zip
Archive:  algoritmos.zip
   creating: exercicios/matriz_99999__assignsubmission_file_/
  inflating: exercicios/matriz_99999__assignsubmission_file_/ALEATORIA.ALG  
  inflating: exercicios/matriz_99999__assignsubmission_file_/INSERCAO.ALG  
  inflating: exercicios/matriz_99999__assignsubmission_file_/MATRIZ.ALG  
  inflating: exercicios/matriz_99999__assignsubmission_file_/MAULTIPLICACAO DE MATRIZES.ALG  
  inflating: exercicios/matriz_99999__assignsubmission_file_/ORDENAMATRIZ.ALG  
  inflating: exercicios/matriz_99999__assignsubmission_file_/PASCAL.ALG  
  inflating: exercicios/matriz_99999__assignsubmission_file_/VENDAS NO ANO.ALG  
   creating: exercicios/outros algoritmos_99999__assignsubmission_file_/
  inflating: exercicios/outros algoritmos_99999__assignsubmission_file_/AREA DO RETANGULO.ALG  
  inflating: exercicios/outros algoritmos_99999__assignsubmission_file_/AREADACIRCUFERENCIA.ALG  
  inflating: exercicios/outros algoritmos_9999

In [365]:
if(modo == 'Upload'):
  alunos = {}
  for folder in os.listdir(pasta_dos_exercicios):
    nome = mask_aluno.findall(folder)
    if (nome):
      nome = nome[0]
      aluno = alunos[nome] = {}

      aluno['caminho'] = pasta_dos_exercicios+'/'+folder
      aluno['exercicios'] = [e for e in [v for v in os.listdir(aluno['caminho'])] if sufixo_arquivo_visgualg == e.lower()[-len(sufixo_arquivo_visgualg):]]

In [366]:
if(modo == 'Upload'):
  pasta_exercicios_c = 'ex_c'
  for nome in alunos:
    aluno = alunos[nome]
    aluno[pasta_exercicios_c] = []
    caminho_ex_c = aluno['caminho']+'/'+pasta_exercicios_c
    try:
      os.mkdir(caminho_ex_c)
    except FileExistsError:
      pass
    for ex in aluno['exercicios']:
      print("Testando o programa <%s>, do aluno <%s>"%(ex, nome))
      programa = VisualG2C(open(aluno['caminho']+'/'+ex, encoding='latin-1').read())
      nome_prog_c = ("%s/%s%s"%(caminho_ex_c,ex.replace(' ','_').replace(' ',sufixo_arquivo_visgualg),sufixo_arquivo_c))
      aluno[pasta_exercicios_c].append(nome_prog_c)
      
      open(nome_prog_c, 'w', encoding='latin-1').write(programa.eval())
      #Compila
      
      #!gcc $nome_prog_c -o teste
      err = os.popen("gcc %s -o teste"%nome_prog_c).read()
      print(err)
      if(err):
        print('\n')
        print("="*40)
        input()
      #Executa
      #!./teste

Testando o programa <PARES.ALG>, do aluno <repeticao>

Testando o programa <MULTIPLOS DE 5.ALG>, do aluno <repeticao>

Testando o programa <EXEMPLO DO ATE FIMREPITA.ALG>, do aluno <repeticao>

Testando o programa <CONTAGEM DE PARES.ALG>, do aluno <repeticao>

Testando o programa <MEDIA.ALG>, do aluno <repeticao>

Testando o programa <POSITIVOS E MEDIA.ALG>, do aluno <repeticao>

Testando o programa <EXEMPLO DO ENQUANTO.ALG>, do aluno <repeticao>

Testando o programa <FATORIAL.ALG>, do aluno <repeticao>

Testando o programa <TABUADA.ALG>, do aluno <repeticao>

Testando o programa <MDC.ALG>, do aluno <repeticao>

Testando o programa <EXEMPLO DO REPITA.ALG>, do aluno <repeticao>

Testando o programa <PERFEITOS.ALG>, do aluno <repeticao>

Testando o programa <TCHAU.ALG>, do aluno <repeticao>

Testando o programa <EXEMPLO DO INTERROMPA.ALG>, do aluno <repeticao>

Testando o programa <PORCENTAGEM DE PARES.ALG>, do aluno <repeticao>

Testando o programa <DESAFIO DO MAIOR NUMERO.ALG>, do aluno

ValueError: ignored

In [ ]:
#files.download(ARQ_NOTAS)